In [1]:
cd /home/humza/Downloads/projects/genml

/home/humza/Downloads/projects/genml


In [3]:
import torch.nn.functional as F

from pathlib import Path
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction
from tqdm import tqdm

from src.genml.data.preprocessors.document_search import create_subdocs
from src.genml.utils.feature_extraction import mean_pooling
from src.genml.constants import (
    INDEX_TRAINING_DOCS_DIRECTORY,
)


In [4]:
model_id = "microsoft/codebert-base"
onnx_path = Path("onnx")
task = "feature-extraction"

# load vanilla transformers and convert to onnx
model = ORTModelForFeatureExtraction.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)



Downloading: 100%|██████████| 498/498 [00:00<00:00, 314kB/s]


In [7]:
subdocs = create_subdocs(INDEX_TRAINING_DOCS_DIRECTORY)
training_data = {
    "embeddings": [],
    "sentences": []
}
for subdoc in tqdm(subdocs):
    try:
        text = subdoc["content"]
        encoded_input = tokenizer(
            text, padding=True, truncation=True, return_tensors="pt"
        )
        model_output = model(**encoded_input)
        sentence_embeddings = mean_pooling(
            model_output, encoded_input["attention_mask"]
        )
        prediction = F.normalize(sentence_embeddings, p=2, dim=1).tolist()
        training_data["embeddings"]+=prediction
        training_data["sentences"].append(text)
    except Exception as e:
        print(e)
        pass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pdftotext version 4.04 [www.xpdfreader.com]
Copyright 1996-2022 Glyph & Cog, LLC


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Syntax Error: Unknown character collection 'Adobe-GB1'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Syntax Error: Unknown character collection 'Adobe-GB1'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1077/1077 [08:48<00:00,  2.04it/s]


In [6]:
training_data

{'embeddings': [[-0.005285919643938541,
   0.0021923885215073824,
   0.010810456238687038,
   0.0081560043618083,
   -0.011831114068627357,
   -0.009073935449123383,
   -0.0025404877960681915,
   0.01221144013106823,
   0.0147726871073246,
   0.01589096523821354,
   -0.007104326970875263,
   0.04360050708055496,
   -0.007602533791214228,
   -0.008007626049220562,
   0.030188973993062973,
   0.0055212476290762424,
   0.0026829810813069344,
   0.016853708773851395,
   -0.004904013127088547,
   -0.006423279643058777,
   -0.00224827928468585,
   -0.009993378072977066,
   0.015542850829660892,
   -0.03126197308301926,
   0.016836533322930336,
   0.009889793582260609,
   0.00590737909078598,
   0.023948391899466515,
   -0.012038973160088062,
   0.04115454852581024,
   0.0008197412244044244,
   -0.0035991331096738577,
   0.09608931094408035,
   -0.0036438796669244766,
   0.01733987033367157,
   -0.016550494357943535,
   -0.008751226589083672,
   0.009205368347465992,
   0.01662690006196499,
 

In [8]:
import os
import json

for key, value in training_data.items():
    filename = os.path.join("data/faiss", f"{key}.json")
    json.dump(value, open(filename, "w"))

In [2]:
from src.genml.utils.faiss_indexing import Faiss


faiss = Faiss(768)

/home/humza/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING clustering 1077 points to 316 centroids: please provide at least 12324 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 1077 points to 256 centroids: please provide at least 9984 trainin

In [14]:
query = "what is transformer?"

encoded_input = tokenizer(
    query, padding=True, truncation=True, return_tensors="pt"
)
model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(
    model_output, encoded_input["attention_mask"]
)
prediction = F.normalize(sentence_embeddings, p=2, dim=1).numpy()
indices = faiss.search(prediction)

In [9]:
import json

sentences = json.load(open("/home/humza/Downloads/projects/genml/data/faiss/sentences.json"))

In [15]:
for index in indices.tolist():
    print(index)

[953, 819, 674, 56, 1076]


In [18]:
close_sentences = [sentences[index-1] for index in indices.tolist()[0]]

In [19]:
close_sentences

['3 We use normalized forms of boxes in our model, hence each value of a box is a float\nbetween 0 and 1. Given two boxes, we sum them after inverse sigmoid and then\ntransform the summation by sigmoid. Refer to Deformable DETR [41] Sec. A.3 for\nmore details. 12 H. Zhang, F. Li, S. Liu, et al. Implementation Details: DINO is composed of a backbone, a Transformer\nencoder, a Transformer decoder, and multiple prediction heads. In appendix D,\nwe provide more implementation details, including all the hyper-parameters and\nengineering techniques used in our models for those who want to reproduce our\nresults.',
 'For span-based reading comprehension, h is passed to a\nfully connection layer to get the start logits s and end log-\nits e of all tokens. The score of a candidate span from po-\nsition i to position j is deﬁned as si\n+ ej, and the maxi-\nmum scoring span where j ≥ i is used as a prediction5. For\nprediction, we compare the score of the pooled ﬁrst token\nspan: snull\n= s0\n+ e